# Ticker Ranking Analysis

Analyze any ticker's position in BUY and SELL setups compared to S&P 500 and NASDAQ 100 indexes.

In [32]:
import sys
from pathlib import Path

# Add src to path
ROOT = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
src_path = ROOT / 'src'
sys.path.insert(0, str(src_path))

from full_scanner import (
    get_sp500_tickers, 
    get_nasdaq100_tickers,
    filter_buy_signals,
    filter_sell_signals,
    calculate_buy_score,
    calculate_sell_score
)
from technical_analysis import analyze_ticker
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

print("✓ Imports loaded")

✓ Imports loaded


## Enter Ticker Symbol

In [33]:
# Enter the ticker to analyze
TICKER = "MSTR"

print(f"Analyzing: {TICKER}")

Analyzing: MSTR


## Analyze Ticker

In [34]:
# Analyze the ticker
print(f"Fetching data for {TICKER}...\n")
ticker_data = analyze_ticker(TICKER, daily_bars=60, weekly_bars=52)

if not ticker_data:
    print(f"❌ Failed to analyze {TICKER}")
else:
    price = ticker_data.get('current_price', 0)
    signal = ticker_data.get('signal', 'N/A')
    
    print(f"{'='*70}")
    print(f"{TICKER} CURRENT ANALYSIS")
    print(f"{'='*70}")
    print(f"Price: ${price:.2f}")
    print(f"Signal: {signal}")
    print(f"")
    print(f"Support S1: ${ticker_data.get('s1', 0):.2f} ({ticker_data.get('s1_quality', 'N/A')})")
    print(f"Resistance R1: ${ticker_data.get('r1', 0):.2f} ({ticker_data.get('r1_quality', 'N/A')})")
    print(f"RSI: {ticker_data.get('rsi', 0):.1f}")
    print(f"{'='*70}\n")

Fetching data for MSTR...

MSTR CURRENT ANALYSIS
Price: $157.33
Signal: CASH

Support S1: $155.61 (CAUTION)
Resistance R1: $200.00 (EXCELLENT)
RSI: 49.2



## Determine Index Membership

In [35]:
# Check which indexes the ticker belongs to
print("Checking index membership...\n")

sp500_tickers = get_sp500_tickers()
nasdaq100_tickers = get_nasdaq100_tickers()

in_sp500 = TICKER in sp500_tickers
in_nasdaq100 = TICKER in nasdaq100_tickers

indexes_to_scan = []
if in_sp500:
    indexes_to_scan.append(('S&P 500', sp500_tickers))
    print(f"✓ {TICKER} is in S&P 500")
else:
    print(f"✗ {TICKER} is NOT in S&P 500")

if in_nasdaq100:
    indexes_to_scan.append(('NASDAQ 100', nasdaq100_tickers))
    print(f"✓ {TICKER} is in NASDAQ 100")
else:
    print(f"✗ {TICKER} is NOT in NASDAQ 100")

if not indexes_to_scan:
    print(f"\n⚠️  {TICKER} is not in S&P 500 or NASDAQ 100 - cannot rank")
else:
    print(f"\nWill rank against: {', '.join([idx[0] for idx in indexes_to_scan])}")

Checking index membership...

[OK] Loaded 503 S&P 500 tickers

[OK] Loaded 101 NASDAQ 100 tickers

✗ MSTR is NOT in S&P 500
✓ MSTR is in NASDAQ 100

Will rank against: NASDAQ 100


## Rank Ticker in Buy and Sell Setups

In [36]:
# Function to calculate R/R ratio for buy setups
def calc_buy_rr(row):
    price = row.get('current_price', 0)
    s1 = row.get('s1', 0)
    r1 = row.get('r1', 0)
    risk = abs((price - s1) / price * 100) if price > 0 and s1 > 0 else 0
    risk = max(risk, 2.0)  # Minimum 2% risk
    reward = ((r1 - price) / price * 100) if price > 0 and r1 > 0 and r1 > price else 0
    return reward / risk if risk > 0 else 0

# Function to calculate R/R ratio for sell setups
def calc_sell_rr(row):
    price = row.get('current_price', 0)
    r1 = row.get('r1', 0)
    s1 = row.get('s1', 0)
    risk = abs((r1 - price) / price * 100) if price > 0 and r1 > 0 and r1 > price else 0
    risk = max(risk, 2.0)  # Minimum 2% risk
    reward = ((price - s1) / price * 100) if price > 0 and s1 > 0 and price > s1 else 0
    return reward / risk if risk > 0 else 0

# Analyze and rank ticker in each index
for index_name, tickers in indexes_to_scan:
    print(f"\n{'='*70}")
    print(f"RANKING IN {index_name}")
    print(f"{'='*70}")
    print(f"Analyzing {len(tickers)} stocks...")
    
    # Analyze all stocks
    results = []
    for i, ticker in enumerate(tickers, 1):
        if i % 50 == 0:
            print(f"  Progress: {i}/{len(tickers)}...")
        try:
            result = analyze_ticker(ticker, daily_bars=60, weekly_bars=52)
            if result:
                results.append(result)
        except:
            pass
    
    df = pd.DataFrame(results)
    print(f"✓ Analyzed {len(df)} stocks successfully\n")
    
    # === BUY SETUPS ===
    print(f"--- BUY SETUP RANKING ---")
    buy_df_all = filter_buy_signals(df, 'FULL HOLD + ADD', quality_filter=False)
    buy_df = filter_buy_signals(df, 'FULL HOLD + ADD', quality_filter=True)
    
    if not buy_df.empty:
        buy_df = buy_df.copy()
        buy_df['score'] = buy_df.apply(calculate_buy_score, axis=1)
        buy_df['rr_ratio'] = buy_df.apply(calc_buy_rr, axis=1)
        buy_df = buy_df.sort_values(['rr_ratio', 'score'], ascending=[False, False])
        buy_df['rank'] = range(1, len(buy_df) + 1)
        
        ticker_buy = buy_df[buy_df['ticker'] == TICKER]
        
        if not ticker_buy.empty:
            row = ticker_buy.iloc[0]
            print(f"✓ {TICKER} qualifies for BUY SETUP")
            print(f"  Rank: #{int(row['rank'])} of {len(buy_df)}")
            print(f"  Score: {row['score']:.0f}/100")
            print(f"  R/R Ratio: 1:{row['rr_ratio']:.1f}")
            print(f"  Signal: {row['signal']}")
            print(f"  Price: ${row['current_price']:.2f}")
            print(f"  Support S1: ${row['s1']:.2f} ({row['buy_quality']})")
            print(f"  Target R1: ${row['r1']:.2f}")
            risk_pct = abs((row['current_price'] - row['s1']) / row['current_price'] * 100)
            risk_pct_adj = max(risk_pct, 2.0)
            reward_pct = ((row['r1'] - row['current_price']) / row['current_price'] * 100) if row['r1'] > row['current_price'] else 0
            print(f"  Risk: {risk_pct:.1f}% (adj: {risk_pct_adj:.1f}%)")
            print(f"  Reward: {reward_pct:.1f}%")
            
            if row['rank'] <= 10:
                print(f"  🎯 IN TOP 10 BUY SETUPS")
            else:
                print(f"  ⚠️  Not in top 10 (needs rank ≤10)")
        else:
            # Check if ticker has FULL HOLD + ADD but excluded by quality
            ticker_buy_all = buy_df_all[buy_df_all['ticker'] == TICKER]
            if not ticker_buy_all.empty:
                row = ticker_buy_all.iloc[0]
                print(f"✗ {TICKER} does NOT qualify for BUY SETUP (excluded by quality)")
                print(f"  Signal: {row['signal']}")
                print(f"  Price: ${row['current_price']:.2f}")
                print(f"  Support S1: ${row['s1']:.2f} (Quality: {row['buy_quality']} ❌)")
                print(f"  Target R1: ${row['r1']:.2f}")
                
                # Calculate metrics anyway
                risk_pct = abs((row['current_price'] - row['s1']) / row['current_price'] * 100)
                risk_pct_adj = max(risk_pct, 2.0)
                reward_pct = ((row['r1'] - row['current_price']) / row['current_price'] * 100) if row['r1'] > row['current_price'] else 0
                rr_ratio = reward_pct / risk_pct_adj if risk_pct_adj > 0 else 0
                score = calculate_buy_score(row)
                
                print(f"  Risk: {risk_pct:.1f}% (adj: {risk_pct_adj:.1f}%)")
                print(f"  Reward: {reward_pct:.1f}%")
                print(f"  R/R Ratio: 1:{rr_ratio:.1f}")
                print(f"  Score: {score:.0f}/100")
                print(f"  Requirement: Quality must be EXCELLENT/GOOD/OK")
            else:
                print(f"✗ {TICKER} does NOT qualify for BUY SETUP")
                print(f"  Current signal: {ticker_data.get('signal')}")
                print(f"  Price: ${ticker_data.get('current_price', 0):.2f}")
                print(f"  Support S1: ${ticker_data.get('s1', 0):.2f} ({ticker_data.get('buy_quality', 'N/A')})")
                print(f"  Target R1: ${ticker_data.get('r1', 0):.2f}")
                print(f"  Requirement: 'FULL HOLD + ADD' signal + EXCELLENT/GOOD/OK quality")
    else:
        # No qualified buy setups, but check if ticker has the signal
        ticker_buy_all = buy_df_all[buy_df_all['ticker'] == TICKER]
        if not ticker_buy_all.empty:
            row = ticker_buy_all.iloc[0]
            print(f"✗ {TICKER} does NOT qualify for BUY SETUP (excluded by quality)")
            print(f"  Signal: {row['signal']}")
            print(f"  Price: ${row['current_price']:.2f}")
            print(f"  Support S1: ${row['s1']:.2f} (Quality: {row['buy_quality']} ❌)")
            print(f"  Target R1: ${row['r1']:.2f}")
            
            risk_pct = abs((row['current_price'] - row['s1']) / row['current_price'] * 100)
            risk_pct_adj = max(risk_pct, 2.0)
            reward_pct = ((row['r1'] - row['current_price']) / row['current_price'] * 100) if row['r1'] > row['current_price'] else 0
            rr_ratio = reward_pct / risk_pct_adj if risk_pct_adj > 0 else 0
            score = calculate_buy_score(row)
            
            print(f"  Risk: {risk_pct:.1f}% (adj: {risk_pct_adj:.1f}%)")
            print(f"  Reward: {reward_pct:.1f}%")
            print(f"  R/R Ratio: 1:{rr_ratio:.1f}")
            print(f"  Score: {score:.0f}/100")
            print(f"  Requirement: Quality must be EXCELLENT/GOOD/OK")
        else:
            print(f"✗ {TICKER} does NOT qualify for BUY SETUP")
            print(f"  Current signal: {ticker_data.get('signal')}")
            print(f"  Price: ${ticker_data.get('current_price', 0):.2f}")
            print(f"  Support S1: ${ticker_data.get('s1', 0):.2f} ({ticker_data.get('buy_quality', 'N/A')})")
            print(f"  Target R1: ${ticker_data.get('r1', 0):.2f}")
            print(f"  Requirement: 'FULL HOLD + ADD' signal + EXCELLENT/GOOD/OK quality")
    
    print()
    
    # === SELL SETUPS ===
    print(f"--- SELL SETUP RANKING ---")
    sell_df = filter_sell_signals(df, quality_filter=False)
    
    if not sell_df.empty:
        sell_df = sell_df.copy()
        sell_df['score'] = sell_df.apply(calculate_sell_score, axis=1)
        sell_df['rr_ratio'] = sell_df.apply(calc_sell_rr, axis=1)
        sell_df = sell_df.sort_values(['rr_ratio', 'score'], ascending=[False, False])
        sell_df['rank'] = range(1, len(sell_df) + 1)
        
        ticker_sell = sell_df[sell_df['ticker'] == TICKER]
        
        if not ticker_sell.empty:
            row = ticker_sell.iloc[0]
            print(f"✓ {TICKER} qualifies for SELL SETUP")
            print(f"  Rank: #{int(row['rank'])} of {len(sell_df)}")
            print(f"  Score: {row['score']:.0f}/100")
            print(f"  R/R Ratio: 1:{row['rr_ratio']:.1f}")
            print(f"  Signal: {row['signal']}")
            print(f"  Price: ${row['current_price']:.2f}")
            print(f"  Resistance R1: ${row['r1']:.2f} ({row['r1_quality']})")
            print(f"  Stop S1: ${row['s1']:.2f}")
            risk_pct = abs((row['r1'] - row['current_price']) / row['current_price'] * 100) if row['r1'] > row['current_price'] else 0
            risk_pct_adj = max(risk_pct, 2.0)
            reward_pct = ((row['current_price'] - row['s1']) / row['current_price'] * 100) if row['current_price'] > row['s1'] else 0
            print(f"  Risk: {risk_pct:.1f}% (adj: {risk_pct_adj:.1f}%)")
            print(f"  Reward: {reward_pct:.1f}%")
            
            if row['rank'] <= 10:
                print(f"  🎯 IN TOP 10 SELL SETUPS")
            else:
                print(f"  ⚠️  Not in top 10 (needs rank ≤10)")
        else:
            # Show ticker info even if it doesn't qualify for sell
            print(f"✗ {TICKER} does NOT qualify for SELL SETUP")
            print(f"  Current signal: {ticker_data.get('signal')}")
            print(f"  Price: ${ticker_data.get('current_price', 0):.2f}")
            print(f"  Resistance R1: ${ticker_data.get('r1', 0):.2f} ({ticker_data.get('r1_quality', 'N/A')})")
            print(f"  Stop S1: ${ticker_data.get('s1', 0):.2f}")
            
            # Calculate metrics
            price = ticker_data.get('current_price', 0)
            r1 = ticker_data.get('r1', 0)
            s1 = ticker_data.get('s1', 0)
            risk_pct = abs((r1 - price) / price * 100) if price > 0 and r1 > 0 and r1 > price else 0
            risk_pct_adj = max(risk_pct, 2.0)
            reward_pct = ((price - s1) / price * 100) if price > 0 and s1 > 0 and price > s1 else 0
            rr_ratio = reward_pct / risk_pct_adj if risk_pct_adj > 0 else 0
            score = calculate_sell_score(ticker_data)
            
            print(f"  Risk: {risk_pct:.1f}% (adj: {risk_pct_adj:.1f}%)")
            print(f"  Reward: {reward_pct:.1f}%")
            print(f"  R/R Ratio: 1:{rr_ratio:.1f}")
            print(f"  Score: {score:.0f}/100")
            print(f"  Requirement: Bearish signal (CASH, REDUCE, etc.)")
    else:
        print(f"No sell setups found in {index_name}")

print(f"\n{'='*70}")
print("ANALYSIS COMPLETE")
print(f"{'='*70}")


RANKING IN NASDAQ 100
Analyzing 101 stocks...
  Progress: 50/101...
  Progress: 100/101...
✓ Analyzed 101 stocks successfully

--- BUY SETUP RANKING ---
✗ MSTR does NOT qualify for BUY SETUP
  Current signal: CASH
  Price: $157.33
  Support S1: $155.61 (CAUTION)
  Target R1: $200.00
  Requirement: 'FULL HOLD + ADD' signal + EXCELLENT/GOOD/OK quality

--- SELL SETUP RANKING ---
✓ MSTR qualifies for SELL SETUP
  Rank: #33 of 51
  Score: 25/100
  R/R Ratio: 1:0.0
  Signal: CASH
  Price: $157.33
  Resistance R1: $200.00 (EXCELLENT)
  Stop S1: $155.61
  Risk: 27.1% (adj: 27.1%)
  Reward: 1.1%
  ⚠️  Not in top 10 (needs rank ≤10)

ANALYSIS COMPLETE
